In [ ]:
import guidance
from guidance import models, gen, one_or_more, select, zero_or_more, regex, optional, capture

model_name = "your_model_name_here"

model = models.LlamaCpp(f"{model_name}.gguf", n_gpu_layers=-1, n_ctx=2048)

llama_new_context_with_model: n_ctx_per_seq (2048) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

In [ ]:
instruction_generate_coffee = """You are a helpful assistant. You have to take as input a customer order and output a list of the corresponding objects. You should use only the following classes in Python:
class Topping:
      def __init__(self, name: str, qualifier: Optional[str] = None, negation: Optional[bool] = False) -> None:

class DrinkOrder:
      def __init__(self, number: int = 1, drink_type: Optional[str] = None, size: Optional[str] = None, style: Optional[str] = None, roast_type: Optional[str] = None, toppings: Optional[List[Topping]] = None) -> None:

The output should be a list of those objects."""

In [ ]:
import json
import spacy
from spacy.pipeline import EntityRuler
from spacy import displacy
import os
import re

def parse_line(line):
    parts = line.strip().split('\t')
    if len(parts) != 2:
        return None, None

    phrase, category = parts

    return phrase, category.strip()

def init_pipeline(dataset = "coffee"):
  nlp = spacy.load("en_core_web_sm")
  ner_ruler = nlp.add_pipe("entity_ruler",
                      before="ner",
                      config={"phrase_matcher_attr": "LOWER"})

  def read_file_categories(food_type):

      file_path = f"FoodOrderingDataset/data/{food_type}/alias"

      text_files = [f for f in os.listdir(file_path) if f.endswith('.txt')]

      patterns = []

      for file in text_files:
          path_to_file = f"{file_path}/{file}"
          with open(path_to_file, 'r') as file:
              for line in file:
                  if line.strip():
                      phrase, category_info = parse_line(line)
                      if phrase and category_info:
                          schema = {}
                          schema["pattern"] = phrase
                          schema["label"] = category_info
                          patterns.append(schema)

      return patterns

  category_patterns = read_file_categories(dataset)

  ner_ruler.add_patterns(category_patterns)
  return nlp


nlp = init_pipeline()

def process_NER(input_order):
    found_categories = []

    doc = nlp(input_order)

    for ent in doc.ents:
        found_categories.append((ent.text, ent.label_))

    return found_categories

In [ ]:
import json
import re
from collections import defaultdict

file_path = f'FoodOrderingDataset/output/Ablation_NER_Coffee_Results_{model_name}.json'

existing_data=[]

with open('FoodOrderingDataset/processed_data/coffee_dataset.json', 'r') as file:
    data = json.load(file)

input_list = []
for obj in data:
    input_value = obj.get("input", "No input key found")
    output_value = obj.get("output_extract", "No output key found")
    output_generate = obj.get("output_generate", "No output key found")
    used_items_value = process_NER(input_value)
    used_items_value_decoupled = [x[0] + ' - ' + x[1] for x in used_items_value]
    used_items_str = ', '.join(used_items_value_decoupled).lower()

    input_augmented_file = input_value + "\nItems Found: " + used_items_str
    input_list.append((input_value, input_augmented_file, output_generate, used_items_value, used_items_str))

for i in range(len(input_list)):
    if i > 130:
        break
    (initial_input, input, expected, used_items_value, used_items_str) = input_list[i]
    lm = model + f'''\
    Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
    ### Instruction:
    {instruction_generate_coffee}
    ### Input:
    {input}

    ### Response:
    '''

    ans = lm + gen("answer", max_tokens=150, stop='\n')

    existing_data.append({"input":initial_input, "input_augmented": input, "output": ans["answer"], "expected":expected, "output_NER": used_items_str})

with open(file_path, 'w') as file:
    json.dump(existing_data, file, indent=4)

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

In [ ]:
import re
from operator import lt, gt

ROAST_CANDIDATES = {"cinnamon"}

def clean_topping(topping_str):
    m = re.match(r"Topping\((.*?)\)", topping_str)
    if not m:
        return topping_str
    inner = m.group(1)
    parts = [p.strip() for p in inner.split(",")]
    params = {}
    for part in parts:
        if "=" in part:
            key, val = part.split("=", 1)
            params[key.strip()] = val.strip()
    for key in list(params.keys()):
        if params[key] == "'None'" or (key == "negation" and params[key] == "False"):
            del params[key]
    if "name" in params and params["name"].strip("'") == "espresso_shot":
        return "Topping(name='ESPRESSO_SHOT_1')"
    if "name" in params:
        return f"Topping(name={params['name']})"
    return topping_str

def canonicalize_drink_order(order_str):
    m = re.match(r"(DrinkOrder)\((.*)\)", order_str)
    if not m:
        return order_str
    order_type, params_str = m.groups()
    params = []
    current = ""
    paren_count = 0
    bracket_count = 0
    for ch in params_str:
        if ch == "(":
            paren_count += 1
        elif ch == ")":
            paren_count -= 1
        elif ch == "[":
            bracket_count += 1
        elif ch == "]":
            bracket_count -= 1
        if ch == "," and paren_count == 0 and bracket_count == 0:
            params.append(current.strip())
            current = ""
        else:
            current += ch
    if current.strip():
        params.append(current.strip())
    
    param_dict = {}
    toppings_list = []
    for param in params:
        if "=" in param:
            key, value = param.split("=", 1)
            key = key.strip()
            value = value.strip()
            if key == "toppings":
                inner = value.strip("[]").strip()
                if inner:
                    tlist = []
                    cur = ""
                    pc = 0
                    for c in inner:
                        if c == "(":
                            pc += 1
                        elif c == ")":
                            pc -= 1
                        if c == "," and pc == 0:
                            tlist.append(cur.strip())
                            cur = ""
                        else:
                            cur += c
                    if cur.strip():
                        tlist.append(cur.strip())
                    for t in tlist:
                        cleaned = clean_topping(t)
                        toppings_list.append(cleaned)
            else:
                param_dict[key] = value
    for key in list(param_dict.keys()):
        if param_dict[key] == "'None'":
            del param_dict[key]
    roast_from_topping = None
    new_toppings = []
    for t in toppings_list:
        m2 = re.match(r"Topping\(name=('[^']+')\)", t)
        if m2:
            name_val = m2.group(1).strip("'")
            if name_val in ROAST_CANDIDATES:
                roast_from_topping = name_val + "_roast"
                continue
        new_toppings.append(t)
    if roast_from_topping and "roast_type" not in param_dict:
        param_dict["roast_type"] = f"'{roast_from_topping}'"
    
    canon_order = ["number", "drink_type", "roast_type", "size", "style", "toppings"]
    new_params = []
    for key in canon_order:
        if key == "toppings":
            if new_toppings:
                new_params.append("toppings=[" + ", ".join(new_toppings) + "]")
        elif key in param_dict:
            new_params.append(f"{key}={param_dict[key]}")
    return f"{order_type}(" + ", ".join(new_params) + ")"

def merge_drink_orders(order_strs):
    orders = []
    for s in order_strs:
        m = re.match(r"DrinkOrder\((.*)\)", s)
        if not m:
            continue
        params_str = m.group(1)
        parts = []
        cur = ""
        bracket_count = 0
        for c in params_str:
            if c == "[":
                bracket_count += 1
            elif c == "]":
                bracket_count -= 1
            if c == "," and bracket_count == 0:
                parts.append(cur.strip())
                cur = ""
            else:
                cur += c
        if cur.strip():
            parts.append(cur.strip())
        d = {}
        for part in parts:
            if "=" in part:
                key, val = part.split("=", 1)
                d[key.strip()] = val.strip()
        orders.append(d)
    
    base = None
    extra = None
    remaining = []
    for o in orders:
        dt = o.get("drink_type", "").strip("'")
        if dt == "espresso":
            extra = o
        else:
            base = o
            remaining.append(o)
    if base and extra:
        def parse_toppings(t_str):
            t_str = t_str.strip("[]")
            if not t_str:
                return []
            return [x.strip() for x in t_str.split(",")]
        base_tops = parse_toppings(base.get("toppings", "[]"))
        extra_tops = parse_toppings(extra.get("toppings", "[]"))
        merged_tops = base_tops + extra_tops
        base["toppings"] = "[" + ", ".join(merged_tops) + "]" if merged_tops else ""
        if "style" not in base and "style" in extra:
            base["style"] = extra["style"]
        merged_order_str = "DrinkOrder(" + ", ".join(f"{k}={v}" for k, v in base.items() if v and v != "'None'") + ")"
        merged = [merged_order_str]
        for o in orders:
            dt_val = o.get("drink_type", "").strip("'")
            if dt_val != "espresso" and o != base:
                merged.append("DrinkOrder(" + ", ".join(f"{k}={v}" for k, v in o.items() if v and v != "'None'") + ")")
        return merged
    else:
        merged = []
        for o in orders:
            merged.append("DrinkOrder(" + ", ".join(f"{k}={v}" for k, v in o.items() if v and v != "'None'") + ")")
        return merged

def split_orders(s):
    s = s.strip()
    if s.startswith("[") and s.endswith("]"):
        s = s[1:-1]
    orders = []
    current = ""
    paren_count = 0
    for char in s:
        if char == "(":
            paren_count += 1
        elif char == ")":
            paren_count -= 1
        if char == "," and paren_count == 0:
            orders.append(current.strip())
            current = ""
        else:
            current += char
    if current.strip():
        orders.append(current.strip())
    
    canon_orders = [canonicalize_drink_order(o) for o in orders]
    merged = merge_drink_orders(canon_orders)
    return "[" + ", ".join(merged) + "]"


[DrinkOrder(number=1, drink_type='latte', roast_type='cinnamon_roast', size='regular', toppings=[Topping(name='ESPRESSO_SHOT_1'), Topping(name='ESPRESSO_SHOT_1')])]


In [ ]:
import json

def calculate_accuracy_and_save_mismatches(json_file, output_file):
    with open(json_file, 'r') as file:
        data = json.load(file)

    total = len(data)
    correct = 0
    mismatches = []

    for item in data:
        output = split_orders(item['output'])
        expected = split_orders(item['expected'])

        if output == expected:
            correct += 1
        else:
            print('\n\n')
            print(output)
            print(expected)
            mismatches.append(item)

    accuracy = (correct / total) * 100

    with open(output_file, 'w') as outfile:
        json.dump(mismatches, outfile, indent=4)

    return accuracy

json_file = f'FoodOrderingDataset/output/Ablation_NER_Coffee_Results_{model_name}.json'
mismatch_file = f'FoodOrderingDataset/output/Ablation_NER_Coffee_Mismatches_{model_name}.json'

accuracy = calculate_accuracy_and_save_mismatches(json_file, mismatch_file)
print(f"Accuracy: {accuracy:.2f}%")
print(f"Mismatches have been saved to: {mismatch_file}")




[]
[DrinkOrder(number=1, drink_type='latte', roast_type='cinnamon_roast', size='regular', style='iced', toppings=[Topping(name='ESPRESSO_SHOT_1')])]



[DrinkOrder(number=1, drink_type='cappuccino', size='large'), DrinkOrder(number=1, drink_type='cappuccino', size='large'), DrinkOrder(number=1, drink_type='cappuccino', size='large'), DrinkOrder(number=1, drink_type='cappuccino', size='large')]
[DrinkOrder(number=1, drink_type='latte', roast_type='light_roast', size='regular', toppings=[Topping(name='ESPRESSO_SHOT_1'), Topping(name='honey')]), DrinkOrder(number=1, drink_type='cappuccino', size='large', toppings=[Topping(name='caramel_syrup')])]



[]
[DrinkOrder(number=1, drink_type='americano', roast_type='dark_roast', size='regular', toppings=[Topping(name='foam')])]



[]
[DrinkOrder(number=1, drink_type='hot_chocolate', size='regular', toppings=[Topping(name='whipped_cream')])]



[]
[DrinkOrder(number=1, drink_type='drip_coffee', roast_type='dark_roast', size='large')]



[Drink